In [4]:
import pandas as pd

df = pd.read_csv('data_tr_apo.csv', encoding= 'cp949')
df.columns= ['date','y']
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M', errors='raise')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['weekday'] = df['date'].dt.weekday

df = df.drop('date', axis = 1)

df2 = pd.read_csv('new_data_ts_apo.csv', encoding = 'cp949')
df2.columns= ['date','y']
df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d %H:%M', errors='raise')
df2['year'] = df2['date'].dt.year
df2['month'] = df2['date'].dt.month
df2['day'] = df2['date'].dt.day
df2['hour'] = df2['date'].dt.hour
df2['weekday'] = df2['date'].dt.weekday

df2 = df2.drop('date', axis = 1)

#이상치 결측값 처리
cond_index = df[(df['y'] >= 1000000) | (df['y'] <= -1000000)].index
for i in cond_index:
    df.loc[i, 'y'] = 0
    
cond_index2 = df[(df['y'] >= 300)].index
for i in cond_index2:
    df.loc[i, 'y'] = 99.205
    
df['y'] = df['y'].fillna(99.205) 

#데이터 셋 준비
y_train = df['y'].copy()
X_train = df[['year','month','day','hour','weekday']].copy()

y_test = df2['y'].copy()
X_test = df2[['year','month','day','hour','weekday']].copy()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

result = []
#mae_lgb = []

In [5]:
for i in range(0,2000): #(8424)까지
    X_train_2017 = X_train[i:336+i]
    X_train_2018 = X_train[8762+i:9098+i]
    X_train_2019 = X_train[17522+i:17858+i]
    X_train_2020 = X_train[26282+i:26618+i]
    
    y_train_2017 = y_train[i:336+i]
    y_train_2018 = y_train[8762+i:9098+i]
    y_train_2019 = y_train[17522+i:17858+i]
    y_train_2020 = y_train[26282+i:26618+i]
    
    X_sum_train = pd.concat([X_train_2017, X_train_2018, X_train_2019, X_train_2020], axis = 0).reset_index(drop = True)
    y_sum_train = pd.concat([y_train_2017, y_train_2018, y_train_2019, y_train_2020], axis = 0).reset_index(drop = True)
    
    X_test_366 = X_test[i+1:337+i].copy()
    y_test_336 = y_test[i+1:337+i].copy()
    
    X_sum_train = X_sum_train.astype('object')
    X_test_366 = X_test_366.astype('object') 

    enc = OneHotEncoder(sparse = False, handle_unknown = 'ignore').fit(X_sum_train)

    X_sum_train_oh = enc.transform(X_sum_train)
    X_test_366_oh = enc.transform(X_test_366)
    
    X_TRAIN = X_sum_train_oh
    X_TEST = X_test_366_oh

    y_TRAIN = y_sum_train.values.ravel()
    y_TEST = y_test_336.values.ravel()
    
    lgb = LGBMRegressor(max_depth = 7,
                    n_estimators = 1000,
                    learning_rate = 0.01,
                    subsample = 0.8,
                    colsample_bytree = 0.8,
                    random_state = 2022)

    model_lgb = lgb.fit(X_TRAIN, y_TRAIN)
    
    pred_final_lgb = model_lgb.predict(X_TEST)
    
    #mae_lgb.append(mean_absolute_error(y_TEST, pred_final_lgb))
    
    result += [pred_final_lgb]

In [6]:
final_result = pd.DataFrame(result)
final_result

,0,1,2,3,4,5,6,7,8,9,...,326,327,328,329,330,331,332,333,334,335
0,64.457822,56.261707,56.011413,53.256268,53.561069,61.068789,74.548012,97.901591,129.331864,138.356677,...,100.171015,95.302327,95.595501,105.279679,130.088349,131.066367,108.397118,100.249526,85.615180,77.525166
1,56.757623,56.724348,53.484370,53.828955,61.232814,74.638121,97.946913,129.705200,138.323753,123.757853,...,94.941548,95.590310,105.405235,130.519865,130.727159,108.395987,100.029097,85.807551,77.561820,65.496188
2,56.478557,53.360693,53.225933,60.979197,74.281359,97.551759,129.580495,137.641286,123.458400,113.970418,...,95.184601,105.110945,130.064817,130.960297,108.603274,100.040941,85.770204,77.053558,65.264545,57.107252
3,52.927496,53.472577,60.918447,73.776398,97.274293,129.652050,137.930347,123.331509,112.529861,123.534738,...,104.814800,129.812646,130.872818,108.393414,99.264080,85.151453,76.992558,65.131249,57.008066,57.542539
4,54.005490,61.646633,74.061437,97.588860,129.780401,138.328370,123.374470,113.315716,124.155572,117.570596,...,129.882653,130.279803,108.724752,99.492198,85.259327,77.371047,65.558168,56.993949,57.508173,53.874704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,44.529037,46.721216,52.513815,91.081712,123.781815,130.969842,128.318250,123.244951,112.389834,117.015915,...,97.162684,131.317955,132.980863,105.528311,93.174381,90.063674,71.193589,66.393821,54.657137,53.082958
1996,47.761677,53.694504,93.205583,124.852741,129.890799,129.296152,126.987470,114.152110,115.472316,104.789942,...,133.430880,136.909219,104.853404,92.605907,87.234204,70.053656,64.231907,53.980269,52.733175,48.633415
1997,52.751876,91.069776,124.635617,131.720367,127.987018,123.189265,112.531564,116.661116,105.725928,95.498000,...,134.287910,105.523061,93.270924,89.565254,70.585983,65.610034,54.112758,52.812205,48.774822,48.405607
1998,92.815810,129.475017,132.382564,128.863668,123.142355,113.628041,117.537660,106.145962,96.582672,96.958019,...,105.313260,93.979921,90.520187,70.077358,65.510147,54.212308,53.034456,48.724181,48.449104,54.824871


In [7]:
final_result.to_csv('그리드_1.csv', index = False)